In [2]:
!pip install django transformers torch pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [ ]:
import os
import django
import json
from django.conf import settings
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from django.core.management import execute_from_command_line
from transformers import pipeline
from pyngrok import ngrok, conf
from django.urls import path


os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


conf.get_default().auth_token = "YOUR_NGROK_AUTH_TOKEN"


if not settings.configured:
    settings.configure(
        DEBUG=True,
        SECRET_KEY="dev",
        ROOT_URLCONF=__name__,
        ALLOWED_HOSTS=["*"],
        MIDDLEWARE=[],
        INSTALLED_APPS=[
            "django.contrib.contenttypes",
            "django.contrib.auth",
        ],
    )

django.setup()


generator = pipeline("text-generation", model="gpt2")

@csrf_exempt
def suggest_titles(request):
    if request.method == "POST":
        try:
            data = json.loads(request.body.decode())
            content = data.get("content", "")
            if not content:
                return JsonResponse({"error": "Content is required."}, status=400)

            prompt = f"Suggest 3 blog titles for the following content:\n{content}\nTitles:"
            output = generator(prompt, max_length=60, num_return_sequences=1)[0]["generated_text"]

            titles = output.split("Titles:")[-1].strip().split("\n")
            titles = [t.strip("-•*1234567890. ").strip() for t in titles if t.strip()]
            return JsonResponse({"titles": titles[:3]})
        except Exception as e:
            return JsonResponse({"error": str(e)}, status=500)

    return JsonResponse({"error": "Only POST requests allowed."}, status=405)


urlpatterns = [path("suggest-titles/", suggest_titles)]

def run():
    port = 8001  
    public_url = ngrok.connect(port)
    print(f"🔗 Public URL: {public_url}")
    print("📮 POST to /suggest-titles/")
    execute_from_command_line(["manage.py", "runserver", f"0.0.0.0:{port}", "--noreload"])

if __name__ == "__main__":
    run()


Device set to use cuda:0


🔗 Public URL: NgrokTunnel: "https://d571-34-59-155-0.ngrok-free.app" -> "http://localhost:8001"
📮 POST to /suggest-titles/
Performing system checks...

System check identified no issues (0 silenced).
April 15, 2025 - 13:42:07
Django version 5.2, using settings None
Starting development server at http://0.0.0.0:8001/
Quit the server with CONTROL-C.

For more information on production servers see: https://docs.djangoproject.com/en/5.2/howto/deployment/
